In [2]:
# This path append is just for testing from within the examples folder, no need for the user to add this
import sys
sys.path.append("..")

CONFIG String

In [9]:
CONFIG_str = """
# SETUP VARIABLES: These determine basic CoMET functionality
verbose: True # Whether to use verbose output
parallel_processing: True # [bool] Whether or not to use parallel processing for certain tasks
max_cores: 32 # Number of cores to use if parallel_processing==True; Enter None for unlimited

# Structered in this form:
# Observation Type:
#   path_to_data
#   additional_observation_parameters
#
#   tracker:
#       tracker_params
#
#       analysis:
#           analysis_variables

# WRF
wrf:
    path_to_data: "/share/D3/data/hweiner/WRF/wrfout_6_19_2013/*"
    
    is_idealized: False

    feature_tracking_var: "dbz" #DBZ, TB, WA, PR, Or other WRF Standard variable name (Case-Sensitive)
    segmentation_var: "dbz"

    tobac:
        feature_id:
            threshold: [30, 40, 50]
            target: "maximum"
            position_threshold: "weighted_diff"
            sigma_threshold: 0.5
            n_min_threshold: 4

        linking:
            method_linking: "predict"
            adaptive_stop: 0.2
            adaptive_step: 0.95
            order: 1
            subnetwork_size: 10
            memory: 0
            v_max: 20
            time_cell_min: 120

        segmentation_2d:
            height: 2 # km
            method: "watershed"
            target: 'maximum'
            threshold: 30

        segmentation_3d:
            method: "watershed"
            target: 'maximum'
            threshold: 30
            seed_3D_flag: 'box'
    
        analysis: # Optional
            merge_split: { variable: "DBZ" }
            eth: { variable: "DBZ", threshold : 30}
            area-low: { height: 2 }
            area-high: { height: 6,}
            volume: {}
            # volume-high: { threshold: 30 }
            max_intensity: { variable : "DBZ"}
            # max_height: { variable : "DBZ", threshold : 30}
            velocity: {}
            perimeter: {}
            cell_growth: {}
            irregularity: {irregularity_metrics: ['convexity', 'sphericity']}

    tams:
        ctt_threshold: 235
        ctt_core_threshold: 219
        u_projection: 0
        parallel: False

        analysis_type: "cloud"

"""

Run CoMET

In [10]:
import CoMET

CONFIG = CoMET.CoMET_load(CONFIG_string=CONFIG_str)
print(CONFIG, "\n")

output = CoMET.CoMET_start(CONFIG=CONFIG)

=====WRF Setup Found in CONFIG=====
{'verbose': True, 'parallel_processing': True, 'max_cores': 32, 'wrf': {'path_to_data': '/share/D3/data/hweiner/WRF/wrfout_6_19_2013/*', 'is_idealized': False, 'feature_tracking_var': 'DBZ', 'segmentation_var': 'DBZ', 'tobac': {'feature_id': {'threshold': [30, 40, 50], 'target': 'maximum', 'position_threshold': 'weighted_diff', 'sigma_threshold': 0.5, 'n_min_threshold': 4}, 'linking': {'method_linking': 'predict', 'adaptive_stop': 0.2, 'adaptive_step': 0.95, 'order': 1, 'subnetwork_size': 10, 'memory': 0, 'v_max': 20, 'time_cell_min': 120}, 'segmentation_2d': {'height': 2, 'method': 'watershed', 'target': 'maximum', 'threshold': 30}, 'segmentation_3d': {'method': 'watershed', 'target': 'maximum', 'threshold': 30, 'seed_3D_flag': 'box'}, 'analysis': {'merge_split': {'variable': 'DBZ'}, 'eth': {'variable': 'DBZ', 'threshold': 30}, 'area-low': {'height': 2}, 'area-high': {'height': 6}, 'volume': {}, 'max_intensity': {'variable': 'DBZ'}, 'velocity': {}, 

=====Performing tobac Segmentation to UDAF=====: 100%|██████████| 25/25 [16:30<00:00, 39.64s/it]


=====Starting WRF tobac Analysis Calculations=====


=====Calculating Cell Sphericities=====: 100%|██████████| 15756/15756 [00:00<00:00, 178028.58it/s]


=====Converting WRF tobac Output to CoMET-UDAF=====
=====WRF tobac Tracking Complete=====
=====Starting WRF TAMS Tracking=====


=====Calculating WRF Brightness Temperatures=====: 100%|██████████| 34687625/34687625 [02:04<00:00, 278236.04it/s]
=====Calculating WRF Precipitation Rate=====: 100%|██████████| 24/24 [00:00<00:00, 143.33it/s]


Starting `identify` 2024-12-04 00:27:40
Starting `track` 2024-12-04 00:27:55
Starting `classify` 2024-12-04 00:27:56
=====Converting TAMS Dataframe to Mask=====


=====Processing TAMS Cell Lifetimes====: 100%|██████████| 103/103 [00:01<00:00, 55.39it/s]


=====WRF TAMS Tracking Complete=====
=====CoMET Performance Diagonistics=====
$ Total Process Time: 34463.65 Seconds
$ Allocated Resources: Cores = 32, Threads = 64


Save Data

In [19]:
import json
import pickle
from copy import deepcopy
import numpy as np

class NpEncoder(json.JSONEncoder):
    def default(self, obj):
        if isinstance(obj, np.integer):
            return int(obj)
        if isinstance(obj, np.floating):
            return float(obj)
        if isinstance(obj, np.ndarray):
            return obj.tolist()
        return super(NpEncoder, self).default(obj)

# Make a copy of the outcopy as to not corrupt it when saving
outcopy = deepcopy(output)

# Define the path where to save the files
savepath = "/share/D3/data/hweiner/CoMET_out/Test_Out"

for dataset in outcopy.keys():

    for tracker in outcopy[dataset].keys():
        print(f"Now saving {dataset} data with {tracker} tracker")

        # Save the tracks and segmentation information
        tracks_copy = deepcopy(outcopy[dataset][tracker]["UDAF_tracks"]) 

        # the csv cannot hold datetime objects, so transform into seconds
        tracks_copy["lifetime"] = tracks_copy["lifetime"].dt.total_seconds()
        tracks_copy.to_csv(f"{savepath}/{dataset.upper()}_Tracks_{tracker}.csv", index = False)

        print("\t Saving tracks information")

        # Save the segmentation if there is any
        if "UDAF_segmentation_3d" in outcopy[dataset][tracker] and outcopy[dataset][tracker]["UDAF_segmentation_3d"] is not None:

            segmentation_3d = deepcopy(outcopy[dataset][tracker]["UDAF_segmentation_3d"])

            with open(f"{savepath}/{dataset.upper()}_segmentation_3d_{tracker}.pickle", "wb") as file:
                pickle.dump(segmentation_3d, file)
            
            print("\t Saving the 3D segmentation information")


        if "UDAF_segmentation_2d" in outcopy[dataset][tracker] and outcopy[dataset][tracker]["UDAF_segmentation_2d"] is not None:

            segmentation_2d = deepcopy(outcopy[dataset][tracker]["UDAF_segmentation_2d"])

            with open(f"{savepath}/{dataset.upper()}_segmentation_2d_{tracker}.pickle", "wb") as file:
                pickle.dump(segmentation_2d, file)

            print("\t Saving the 2D segmentation information")


        # Save the analysis data in a dictionary
        if "analysis" in outcopy[dataset][tracker] and len(outcopy[dataset][tracker]["analysis"]) != 0:

            print(f"\t Now saving analysis variables:")

            save_dict = {}

            for analysis_variable in outcopy[dataset][tracker]["analysis"].keys():

                if analysis_variable in ["area-high", "area-low"]:

                    save_dict[f"{dataset}_{tracker}_{analysis_variable}"] = outcopy[dataset][tracker]["analysis"][analysis_variable]["area"].values.tolist()
                    print(f"\t \t {analysis_variable}")

                elif analysis_variable in ["irregularity"]:
                    
                    # go through all of the irregularity measurements in analysis
                    for irregularity_key in outcopy[dataset][tracker]["analysis"][analysis_variable].keys()[3:]: # skip frame, feature id, cell id
                        
                        save_dict[f"{dataset}_{tracker}_{irregularity_key}"] = outcopy[dataset][tracker]["analysis"][analysis_variable][irregularity_key].values.tolist()
                        print(f"\t \t {irregularity_key}")

                elif analysis_variable in ["velocity"]:
                    
                    # save both the speed and velocity from the velocity analysis dataframe
                    save_dict[f"{dataset}_{tracker}_velocity"] = outcopy[dataset][tracker]["analysis"][analysis_variable]["velocity"].values.tolist()
                    save_dict[f"{dataset}_{tracker}_speed"] = outcopy[dataset][tracker]["analysis"][analysis_variable]["speed"].values.tolist()
                    print(f"\t \t speed \n\t \t velocity")

                elif analysis_variable == "merge_split":
                    
                    save_dict[f"{dataset}_{tracker}_merger_frame"] = outcopy[dataset][tracker]["analysis"][analysis_variable][0]["frame"].values.tolist()
                    save_dict[f"{dataset}_{tracker}_merger_parent_cells"] = outcopy[dataset][tracker]["analysis"][analysis_variable][0]["parent_cells"].values.tolist()
                    save_dict[f"{dataset}_{tracker}_merger_merged_cell"] = outcopy[dataset][tracker]["analysis"][analysis_variable][0]["merged_cell"].values.tolist()

                    save_dict[f"{dataset}_{tracker}_splitter_frame"] = outcopy[dataset][tracker]["analysis"][analysis_variable][1]["frame"].values.tolist()
                    save_dict[f"{dataset}_{tracker}_splitter_split_cell"] = outcopy[dataset][tracker]["analysis"][analysis_variable][1]["split_cell"].values.tolist()
                    save_dict[f"{dataset}_{tracker}_splitter_child_cells"] = outcopy[dataset][tracker]["analysis"][analysis_variable][1]["child_cells"].values.tolist()
                    print("\t \t merge/split information")

                else:
                    
                    save_dict[f"{dataset}_{tracker}_{analysis_variable}"] = outcopy[dataset][tracker]["analysis"][analysis_variable][analysis_variable].values.tolist()
                    print(f"\t \t {analysis_variable}")

            with open(f"{savepath}/{dataset.upper()}_AnalysisVariables_{tracker}.json", "w") as outfile:
                json.dump(save_dict, outfile, cls=NpEncoder)

Now saving wrf data with tobac tracker
	 Saving tracks information
	 Saving the 3D segmentation information
	 Saving the 2D segmentation information
	 Now saving analysis variables:
	 	 eth
	 	 perimeter
	 	 volume
	 	 merge/split information
	 	 area-low
	 	 area-high
	 	 max_intensity
	 	 speed 
	 	 velocity
	 	 cell_growth
	 	 convexity
	 	 sphericity
Now saving wrf data with tams tracker
	 Saving tracks information
	 Saving the 2D segmentation information


Open Data

In [22]:
import pandas as pd
import json
import pickle

path_to_files = "/share/D3/data/hweiner/CoMET_out/Test_Out"

with open(f"{path_to_files}/WRF_AnalysisVariables_tobac.json", "r") as file:
    WRF_analysis_data_tobac = json.load(file)

WRF_tracks_tobac = pd.read_csv(f"{path_to_files}/WRF_Tracks_tobac.csv")
WRF_tracks_tams = pd.read_csv(f"{path_to_files}/WRF_Tracks_tams.csv")

with open(f"{path_to_files}/WRF_segmentation_2d_tobac.pickle", "rb") as data:
    WRF_segmentation_2d_tobac = pickle.load(data)
with open(f"{path_to_files}/WRF_segmentation_2d_tams.pickle", "rb") as data:
    WRF_segmentation_2d_tams = pickle.load(data)

In [ ]:
WRF_analysis_data_tobac

In [ ]:
WRF_tracks_tobac

In [ ]:
WRF_segmentation_2d_tobac